# Task

При помощи любой доступной Вам LLM (open-source, проприетарной) создать прототип решения, которое автоматизировало бы:

- Шаг 1. Дополнение QR-меню ресторана информацией с описанием блюд/напитков, которое будет cгенерировано LLM. По каждому блюду/напитку к уже имеющимся данным о названии и цене должен быть добавлен пункт «description», содержащий короткое описание данного блюда/напитка (не более 350 символов);

- Шаг 2. Перевод QR-меню ресторана на английский язык.

Дано: JSON-файл, содержащий меню ресторана на испанском языке. Блюда и напитки в меню разбиты по категориям.

Результат: валидный JSON-файл, содержащий меню в исходной разбивке по категориям блюд, и при этом дополненный описаниями блюд и переведенный на английский язык.


Результат необходимо сопроводить следующей информацией:
- Описание выполненной работы (какая конфигурация решения использована: LLM, библиотеки, др.);
- Текст использованных промптов;
- Cопутствующая аналитика: что получилось, что нет, какие есть пути повышения качества решения задачи.

# Solution 1

## Description

Самым простым подходом к решению данной задачи было бы написать тривиальный промпт к любой проприетарной LLM, формата: 
    
    "You will be provided with name of the dish from the menu and the name of the dish category. You have to respond with a concise description of the dish in Spanish. The description must not exceed 350 characters in total."
    
Для перевода можно воспользоваться любым проприетарным переводчиком yandex/google/deepl.

## Implementation

### Imports

In [20]:
import os
import json
import backoff
import requests

from typing import Optional
from dotenv import load_dotenv
from tqdm.notebook import tqdm
from pydantic_settings import BaseSettings
from openai import OpenAI, APIConnectionError, RateLimitError

### Config

In [33]:
load_dotenv()

class Settings(BaseSettings):
    ## File paths
    FILE_PATH: str = "/mnt/c/Users/Games/Desktop/beep-dev/prompting-test/data/JSON_input_Viva_Tikila.json"
    RESULT_DESCRIPTIONS_FILE_PATH: str = "/mnt/c/Users/Games/Desktop/beep-dev/prompting-test/data/solution_1_descriptions.json"
    RESULT_TRANSLATION_FILE_PATH: str = "/mnt/c/Users/Games/Desktop/beep-dev/prompting-test/data/solution_1_translation.json"

    ## File fields
    CATEGORIES_LIST_FIELD: str = "categories"
    ITEMS_LIST_FIELD: str = "items"
    CATEGORY_FIELD: str = "category"
    ITEM_NAME_FIELD: str = "name"
    ITEM_DESCRIPTION_FIELD: str = "description"

    ## LLM
    #LLM_URL: str = os.environ["LLM_URL"] # for self-host/open-router/etc openai-compatible LLM
    LLM_API_KEY: str = os.environ["LLM_API_KEY"]
    LLM_NAME: str = "gpt-4o-mini"
    MAX_TOKENS: int = 100
    MAX_WAIT_TIME: int = 60  # больше 60 бессмысленно, кд на TPM пройдет

    ## Translate
    TRANSLATION_URL: str = os.environ["TRANSLATION_URL"]
    TRANSLATION_API_KEY: str = os.environ["TRANSLATION_API_KEY"]
    YANDEX_FOLDER_ID: str = os.environ["YANDEX_FOLDER_ID"]

    ## Prompts
    DESCRIPTION_GENERATION_SYSTEM_PROMPT: str = """You are a helpful culinary assistant. You will be provided with name of the dish from the menu and the name of the dish category. 
    You have to respond with a concise description of the dish in Spanish. The description must not exceed 350 characters in total. Return result in a valid json: {"description": <GENERATED_DESCRIPTION>}
    """
    DESCRIPTION_GENERATION_PROMPT: str = """Generate a short Spanish description (no more than 350 characters) for the following dish with category: "{category}" and name: "{name}"."""

settings = Settings()



### Utils

In [9]:
def import_data(file_path: str) -> Optional[dict]:
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except PermissionError:
        print(f"Error: Permission denied while trying to read the file '{file_path}'.")
    except json.JSONDecodeError as e:
        print(f"Error: Failed to decode JSON. Check the file for syntax errors. Details: {e}")
    except Exception as e:
        print(f"Unexpected error occurred: {e}")
    return None

In [10]:
def validate_data_structure(data: dict) -> bool:
    """
    Validate the structure of a JSON object against the expected format.

    Parameters:
        data (dict): The JSON object to validate.

    Returns:
        bool: True if the JSON structure is valid, False otherwise.
    """

    if settings.CATEGORIES_LIST_FIELD not in data:
        print(f"Error: Missing top-level field '{settings.CATEGORIES_LIST_FIELD}'.")
        return False

    if not isinstance(data[settings.CATEGORIES_LIST_FIELD], list):
        print(f"Error: '{settings.CATEGORIES_LIST_FIELD}' should be a list.")
        return False

    for category in data[settings.CATEGORIES_LIST_FIELD]:
        if not isinstance(category, dict):
            print("Error: Each category should be a dictionary.")
            return False

        if settings.CATEGORY_FIELD not in category:
            print(f"Error: Missing field '{settings.CATEGORY_FIELD}' in a category.")
            return False

        if settings.ITEMS_LIST_FIELD not in category:
            print(f"Error: Missing field '{settings.ITEMS_LIST_FIELD}' in category '{category}'.")
            return False

        if not isinstance(category[settings.ITEMS_LIST_FIELD], list):
            print(f"Error: '{settings.ITEMS_LIST_FIELD}' should be a list in category '{category[settings.CATEGORY_FIELD]}'.")
            return False

        for item in category[settings.ITEMS_LIST_FIELD]:
            if not isinstance(item, dict):
                print(f"Error: Each item in '{settings.ITEMS_LIST_FIELD}' should be a dictionary.")
                return False

            if settings.ITEM_NAME_FIELD not in item:
                print(f"Error: Missing field '{settings.ITEM_NAME_FIELD}' in an item of category '{category[settings.CATEGORY_FIELD]}'.")
                return False

    return True

In [11]:
def get_openai_client() -> OpenAI:
    client = OpenAI(api_key=settings.LLM_API_KEY)
    return client

def backoff_hdlr(details: dict) -> None:
    print(f"Backing off {details['wait']:0.1f} seconds after {details['tries']} tries ")


In [12]:
@backoff.on_exception(
    backoff.expo,
    (RateLimitError, APIConnectionError),
    max_tries=5,
    max_time=settings.MAX_WAIT_TIME,
    jitter=backoff.full_jitter,
    on_backoff=backoff_hdlr,
)
def generate_response(prompt: str, system_prompt: str, openai_client: OpenAI) -> str:
    response = openai_client.chat.completions.create(
        model=settings.LLM_NAME,
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ],
        max_tokens=settings.MAX_TOKENS,
    )
    return response.choices[0].message.content

In [14]:
def save_json(data: dict, output_file_path: str) -> None:
    """
    Save data to a JSON file with exception handling.

    Parameters:
        data (dict): The data to save.
        output_file_path (str): The file path where the JSON should be saved.

    Raises:
        Exception: If any error occurs while saving the file.
    """
    try:
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            json.dump(data, output_file, ensure_ascii=False, indent=2)
        print(f"Data successfully saved to {output_file_path}")
    except FileNotFoundError:
        print(f"Error: The specified path '{output_file_path}' does not exist.")
    except PermissionError:
        print(f"Error: Permission denied when trying to write to '{output_file_path}'.")
    except Exception as e:
        print(f"An unexpected error occurred while saving the JSON: {e}")
        raise Exception("Unexpected error in save_json")

In [38]:
def translate(text: str, target_language: str = "en", source_language: str = "es") -> str:
    body = {
    "sourceLanguageCode": source_language,
    "targetLanguageCode": target_language,
    "texts": [text],
    "folderId": settings.YANDEX_FOLDER_ID,
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key {0}".format(settings.TRANSLATION_API_KEY)
    }

    response = requests.post(settings.TRANSLATION_URL,
        json = body,
        headers = headers
    )

    return json.loads(response.text)["translations"][0]["text"]

### Main (generate descriptions)

In [15]:
data = import_data(settings.FILE_PATH)

if not validate_data_structure(data=data):
    raise Exception("Please fix identified errors in the provided file.")

openai_client = get_openai_client()

categories = data.get(settings.CATEGORIES_LIST_FIELD, [])
for category in tqdm(categories, desc="Processing Categories"):
    category_name = category[settings.CATEGORY_FIELD]
    items = category.get(settings.ITEMS_LIST_FIELD, [])
    for item in tqdm(items, desc=f"Processing Items in {category_name}", leave=False):
        item_name = item[settings.ITEM_NAME_FIELD]
        try:
            description_json = generate_response(
                prompt=settings.DESCRIPTION_GENERATION_PROMPT.format(category=category_name, name=item_name),
                system_prompt=settings.DESCRIPTION_GENERATION_SYSTEM_PROMPT,
                openai_client=openai_client
            )

            description = json.loads(description_json)["description"]
        except (KeyError, json.JSONDecodeError) as e:
            print(f"Error processing item '{item_name}' in category '{category_name}': {e}")
            description = "Description unavailable"
        except Exception as e:
            print(f"Error processing item '{item_name}' in category '{category_name}': {e}")
            description = "Description unavailable"

        item[settings.ITEM_DESCRIPTION_FIELD] = description
        
save_json(data=data, output_file_path=settings.RESULT_DESCRIPTIONS_FILE_PATH)

Processing Categories:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Items in Cafés:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Items in Bebidas frías:   0%|          | 0/4 [00:00<?, ?it/s]

Processing Items in Jugos Frescos:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Items in Smoothies:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Items in Desayunos:   0%|          | 0/8 [00:00<?, ?it/s]

Processing Items in A cualquier hora:   0%|          | 0/14 [00:00<?, ?it/s]

Data successfully saved to /mnt/c/Users/Games/Desktop/beep-dev/prompting-test/data/solution_1_descriptions.json


### Main (translate)

In [39]:
def translate_data(data, target_language="en"):
    categories = data.get(settings.CATEGORIES_LIST_FIELD, [])
    for category in tqdm(categories, desc="Processing Categories"):
        category_name = category[settings.CATEGORY_FIELD]
        category[settings.CATEGORY_FIELD] = translate(category_name, target_language)

        items = category.get(settings.ITEMS_LIST_FIELD, [])
        for item in tqdm(items, desc=f"Processing Items in {category_name}", leave=False):
            item[settings.ITEM_NAME_FIELD] = translate(item[settings.ITEM_NAME_FIELD], target_language)
            item[settings.ITEM_DESCRIPTION_FIELD] = translate(item[settings.ITEM_DESCRIPTION_FIELD], target_language)

    return data

translated_data = translate_data(data, target_language="en")
save_json(data=translated_data, output_file_path=settings.RESULT_TRANSLATION_FILE_PATH)

Processing Categories:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Items in Coffee Shops:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Items in Cold drinks:   0%|          | 0/4 [00:00<?, ?it/s]

Processing Items in Fresh Juices:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Items in Smoothies:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Items in Breakfasts:   0%|          | 0/8 [00:00<?, ?it/s]

Processing Items in At any time:   0%|          | 0/14 [00:00<?, ?it/s]

Data successfully saved to /mnt/c/Users/Games/Desktop/beep-dev/prompting-test/data/solution_1_translation.json


## Conclusion
Подход, в общем и целом, рабочий, но сразу видны потенциальные минусы:
1. слабая предсказуемость результатов: 
    - различные непредсказуемые стили описания блюд
    - нет гарантии, что модель ответит в соответствии с заданными ограничениями (<=350 символов), тк она оперирует токенами 🍓
2. галюцинации, если наименование блюда слишком творческое/недостаточно информативное, модель может словить галюнов и выдать некорректное описание
3. проблемы перевода, при ручном отсмотре итогового файла было выявлено три однотипные ошибки, при раздельном переводе названия позиции и ее описания, название может быть переведено корректно в соответствующем поле, однако будет оставлен оригинал названия позиции в описании (или наоборот):
    - {"name": "Papas mix (French fries, Egg, Salad)", "description": "Mix Potatoes are a ..."}
    - {"name": "Chilaquiles with Chicken", "description": "Chilaquiles con pollo are a ..."}
    - {"name": "Chilaquiles with Egg", "description": "Chilaquiles con huevo are a ..."}


# Improvements
Итак, давайте разберем как можно решить каждую из проблем
1. для получения более предсказуемых результатов:
    - используем few-shot подход для генерации описаний (возможно у ресторана семейная тематика и владелец хочет сохранить посыл в описаниях блюд)
    - добавляем программный контроль количества символов в ответе LLM
2. можно придумать множество способов защититься от таких ошибок, самым простым из которых будет добавление соответствующих инструкций в промпт и подобных примеров в few-shot
3. для сохранения консистентности названий товаров в полях описаний и наименований, самым простым является одновременный перевод всех текстовых полей позиции меню с использованием LLM. при том, перевод названий секций меню можно оставить через переводчик, так как проблем с их переводом при тестировании выявлено не было (да и не должно возникнуть в будущем)

# Solution 2

## Description

Большая часть кода будет полностью идентична первому решению, поэтому ниже представлены только обновленные/добавленные фрагменты.
В данном решении тут будут улучшенные промпты, валидация ответа модели (на предмет соответствия условию по длине) и перевод позиций меню с использованием LLM.

В соответствии с улучшениями, предложенными выше, скорректируем промпты и подход к генерации описаний и переводу меню
### Промпт для генерации описаний будет иметь примерно такой вид:

>    You are a helpful culinary assistant. You will be provided with the name of a dish from the menu and the name of the dish category. 
>    Your task is to respond with a concise description of the dish in Spanish. The description must align with a family restaurant style and must not exceed 350 characters in total.
>
>    If the name of the dish does not provide enough meaningful information to generate a description or is unclear, respond with: {"description": "Unpredictable"}.
>
>    Return the result in a valid JSON format: {"description": <GENERATED_DESCRIPTION>}.
>
>    Here are some examples to guide you:
>
>    Category: "Postres"<br>
>    Name: "Pastel Tres Leches"<br>
>    Description: {"description": "Un esponjoso pastel bañado en tres tipos de leche, coronado con crema batida, perfecto para los amantes de los postres dulces y cremosos."}
>
>    Category: "Entradas"<br>
>    Name: "Guacamole con Totopos"<br>
>    Description: {"description": "Guacamole fresco hecho con aguacates maduros, jugo de limón y un toque de cilantro, acompañado de crujientes totopos de maíz."}
>
>    Category: "Sopas"<br>
>    Name: "Sopa de Tortilla"<br>
>    Description: {"description": "Tradicional sopa mexicana con tiras de tortilla crujiente, aguacate, queso fresco y un toque de chile pasilla."}
>
>    Category: "A cualquier hora"<br>
>    Name: "Flambe Bambe"<br>
>    Description: {"description": "Unpredictable"}"
>
>    Category: "Ensaladas"<br>
>    Name: "Ensalada César"<br>
>    Description: {"description": "Crujientes hojas de lechuga romana con aderezo César, crotones y queso parmesano fresco."}
>
>    Category: "Tacos"<br>
>    Name: "Tacos de Barbacoa"<br>
>    Description: {"description": "Tacos suaves con carne de barbacoa cocida lentamente, acompañados de cebolla y salsa verde."}
>
>    Category: "Smoothies"<br>
>    Name: "Caribe (plátano, coco, chocolate)"<br>
>    Description: {"description": "Una mezcla exótica y dulce de plátano, coco y chocolate, perfecta para consentirte."}
>
>    Category: "Café"<br>
>    Name: "Mocha"<br>
>    Description: {"description": "Una deliciosa combinación de café espresso, chocolate y leche cremosa, perfecta para los amantes del chocolate"}
>
>    Category: "Desayunos"<br>
>    Name: "Huevos Motuleños (2 huevos fritos sobre tortilla, salsa roja, jamón y plátano frito)"<br>
>    Description: {"description": "Un clásico desayuno yucateco con huevos fritos sobre tortilla, salsa roja, jamón y un toque dulce de plátano frito."}
>
>    Category: "Café"<br>
>    Name: "Bloom"<br>
>    Description: {"description": "Unpredictable"}


Перевод наименований и описаний также будет произведен с использованием LLM, для управления стилем перевода, аналогично промпту выше, здесь использован few-shot подход.
### Промпт для перевода будет иметь примерно такой вид:
>    You are a helpful culinary assistant. You will be provided with name, and description of a dish from a menu in Spanish. 
>    Your task is to translate the name and description into English, preserving the original meaning, style, and tone.
>
>    Return the result in a valid JSON format: {"name": <TRANSLATED_NAME>, "description": <TRANSLATED_DESCRIPTION>}.
>
>    Here are some examples to guide you:
>
>    Name: "Pastel Tres Leches"<br>
>    Description: {"description": "Un esponjoso pastel bañado en tres tipos de leche, coronado con crema batida, perfecto para los amantes de los postres dulces y cremosos."}<br>
>    Translation: {"name": "Three Milk Cake", "description": "A fluffy cake soaked in three types of milk, topped with whipped cream, perfect for lovers of sweet and creamy desserts."}
>
>    Name: "Guacamole con Totopos"<br>
>    Description: {"description": "Guacamole fresco hecho con aguacates maduros, jugo de limón y un toque de cilantro, acompañado de crujientes totopos de maíz."}<br>
>    Translation: {"name": "Guacamole with Tortilla Chips", "description": "Fresh guacamole made with ripe avocados, lime juice, and a touch of cilantro, served with crispy tortilla chips."}
>
>    Name: "Sopa de Tortilla"<br>
>    Description: {"description": "Tradicional sopa mexicana con tiras de tortilla crujiente, aguacate, queso fresco y un toque de chile pasilla."}<br>
>    Translation: {"name": "Tortilla Soup", "description": "Traditional Mexican soup with crispy tortilla strips, avocado, fresh cheese, and a hint of pasilla chili."}
>
>    Name: "Ensalada César"<br>
>    Description: {"description": "Crujientes hojas de lechuga romana con aderezo César, crotones y queso parmesano fresco."}<br>
>    Translation: {"name": "Caesar Salad", "description": "Crisp romaine lettuce with Caesar dressing, croutons, and fresh Parmesan cheese."}
>
>    Name: "Tacos de Barbacoa"<br>
>    Description: {"description": "Tacos suaves con carne de barbacoa cocida lentamente, acompañados de cebolla y salsa verde."}<br>
>    Translation: {"name": "Barbacoa Tacos", "description": "Soft tacos filled with slow-cooked barbacoa meat, served with onions and green salsa."}
>
>    Name: "Caribe (plátano, coco, chocolate)"<br>
>    Description: {"description": "Una mezcla exótica y dulce de plátano, coco y chocolate, perfecta para consentirte."}<br>
>    Translation: {"name": "Caribbean (banana, coconut, chocolate)", "description": "An exotic and sweet blend of banana, coconut, and chocolate, perfect for indulging yourself."}
>
>    Name: "Mocha"<br>
>    Description: {"description": "Una deliciosa combinación de café espresso, chocolate y leche cremosa, perfecta para los amantes del chocolate"}<br>
>    Translation: {"name": "Mocha", "description": "A delightful combination of espresso coffee, chocolate, and creamy milk, perfect for chocolate lovers."}
>
>    Name: "Huevos Motuleños (2 huevos fritos sobre tortilla, salsa roja, jamón y plátano frito)"<br>
>    Description: {"description": "Un clásico desayuno yucateco con huevos fritos sobre tortilla, salsa roja, jamón y un toque dulce de plátano frito."}<br>
>    Translation: {"name": "Motuleños Eggs (2 fried eggs on tortilla, red sauce, ham, and fried plantain)", "description": "A classic Yucatecan breakfast with fried eggs on tortilla, red sauce, ham, and a sweet touch of fried plantain."}

## Implementation

### Imports

In [40]:
import os
import json
import backoff
import requests

from typing import Optional
from dotenv import load_dotenv
from tqdm.notebook import tqdm
from pydantic_settings import BaseSettings
from openai import OpenAI, APIConnectionError, RateLimitError

### Config

In [69]:
load_dotenv()

class Settings(BaseSettings):
    ## File paths
    FILE_PATH: str = "/mnt/c/Users/Games/Desktop/beep-dev/prompting-test/data/JSON_input_Viva_Tikila.json"
    RESULT_DESCRIPTIONS_FILE_PATH: str = "/mnt/c/Users/Games/Desktop/beep-dev/prompting-test/data/solution_2_descriptions.json"
    RESULT_TRANSLATION_FILE_PATH: str = "/mnt/c/Users/Games/Desktop/beep-dev/prompting-test/data/solution_2_translation.json"

    ## File fields
    CATEGORIES_LIST_FIELD: str = "categories"
    ITEMS_LIST_FIELD: str = "items"
    CATEGORY_FIELD: str = "category"
    ITEM_NAME_FIELD: str = "name"
    ITEM_DESCRIPTION_FIELD: str = "description"

    ## LLM
    #LLM_URL: str = os.environ["LLM_URL"] # for self-host/open-router/etc openai-compatible LLM
    LLM_API_KEY: str = os.environ["LLM_API_KEY"]
    LLM_NAME: str = "gpt-4o-mini"
    MAX_TOKENS: int = 100
    MAX_WAIT_TIME: int = 60  # больше 60 бессмысленно, кд на TPM пройдет
    MAX_RESPONSE_LENGTH: int = 350

    ## Translate
    TRANSLATION_URL: str = os.environ["TRANSLATION_URL"]
    TRANSLATION_API_KEY: str = os.environ["TRANSLATION_API_KEY"]
    YANDEX_FOLDER_ID: str = os.environ["YANDEX_FOLDER_ID"]

    ## Prompts
    DESCRIPTION_GENERATION_SYSTEM_PROMPT: str = """You are a helpful culinary assistant. You will be provided with the name of a dish from the menu and the name of the dish category. 
    Your task is to respond with a concise description of the dish in Spanish. The description must align with a family restaurant style and must not exceed 350 characters in total.

    If the name of the dish does not provide enough meaningful information to generate a description or is unclear, respond with: {"description": "Unpredictable"}.

    Return the result in a valid JSON format: {"description": <GENERATED_DESCRIPTION}.

    Here are some examples to guide you:

    Category: "Postres"
    Name: "Pastel Tres Leches"
    Description: {"description": "Un esponjoso pastel bañado en tres tipos de leche, coronado con crema batida, perfecto para los amantes de los postres dulces y cremosos."}

    Category: "Entradas"
    Name: "Guacamole con Totopos"
    Description: {"description": "Guacamole fresco hecho con aguacates maduros, jugo de limón y un toque de cilantro, acompañado de crujientes totopos de maíz."}

    Category: "Sopas"
    Name: "Sopa de Tortilla"
    Description: {"description": "Tradicional sopa mexicana con tiras de tortilla crujiente, aguacate, queso fresco y un toque de chile pasilla."}

    Category: "A cualquier hora"
    Name: "Carne"
    Description: {"description": "Unpredictable"}"

    Category: "Ensaladas"
    Name: "Ensalada César"
    Description: {"description": "Crujientes hojas de lechuga romana con aderezo César, crotones y queso parmesano fresco."}

    Category: "Tacos"
    Name: "Tacos de Barbacoa"
    Description: {"description": "Tacos suaves con carne de barbacoa cocida lentamente, acompañados de cebolla y salsa verde."}

    Category: "Smoothies"
    Name: "Caribe (plátano, coco, chocolate)"
    Description: {"description": "Una mezcla exótica y dulce de plátano, coco y chocolate, perfecta para consentirte."}

    Category: "Café"
    Name: "Mocha"
    Description: {"description": "Una deliciosa combinación de café espresso, chocolate y leche cremosa, perfecta para los amantes del chocolate"}

    Category: "Desayunos"
    Name: "Huevos Motuleños (2 huevos fritos sobre tortilla, salsa roja, jamón y plátano frito)"
    Description: {"description": "Un clásico desayuno yucateco con huevos fritos sobre tortilla, salsa roja, jamón y un toque dulce de plátano frito."}

    Category: "Café"
    Name: "Culo Llameante"
    Description: {"description": "Unpredictable"}"""

    DESCRIPTION_GENERATION_PROMPT: str = """Generate a short Spanish description (no more than 350 characters) for the following dish with category: "{category}" and name: "{name}".
    If the name of the dish is unclear or does not provide enough information for a meaningful description, return: {{"description": "Unpredictable"}}.
    """

    TRANSLATION_SYSTEM_PROMPT: str = """You are a helpful culinary assistant. You will be provided with name, and description of a dish from a menu in Spanish. 
    Your task is to translate the name and description into English, preserving the original meaning, style, and tone.

    Return the result in a valid JSON format: {"name": <TRANSLATED_NAME, "description": <TRANSLATED_DESCRIPTION}.

    Here are some examples to guide you:

    Name: "Pastel Tres Leches"
    Description: {"description": "Un esponjoso pastel bañado en tres tipos de leche, coronado con crema batida, perfecto para los amantes de los postres dulces y cremosos."}
    Translation: {"name": "Three Milk Cake", "description": "A fluffy cake soaked in three types of milk, topped with whipped cream, perfect for lovers of sweet and creamy desserts."}

    Name: "Guacamole con Totopos"
    Description: {"description": "Guacamole fresco hecho con aguacates maduros, jugo de limón y un toque de cilantro, acompañado de crujientes totopos de maíz."}
    Translation: {"name": "Guacamole with Tortilla Chips", "description": "Fresh guacamole made with ripe avocados, lime juice, and a touch of cilantro, served with crispy tortilla chips."}

    Name: "Sopa de Tortilla"
    Description: {"description": "Tradicional sopa mexicana con tiras de tortilla crujiente, aguacate, queso fresco y un toque de chile pasilla."}
    Translation: {"name": "Tortilla Soup", "description": "Traditional Mexican soup with crispy tortilla strips, avocado, fresh cheese, and a hint of pasilla chili."}

    Name: "Ensalada César"
    Description: {"description": "Crujientes hojas de lechuga romana con aderezo César, crotones y queso parmesano fresco."}
    Translation: {"name": "Caesar Salad", "description": "Crisp romaine lettuce with Caesar dressing, croutons, and fresh Parmesan cheese."}

    Name: "Tacos de Barbacoa"
    Description: {"description": "Tacos suaves con carne de barbacoa cocida lentamente, acompañados de cebolla y salsa verde."}
    Translation: {"name": "Barbacoa Tacos", "description": "Soft tacos filled with slow-cooked barbacoa meat, served with onions and green salsa."}

    Name: "Caribe (plátano, coco, chocolate)"
    Description: {"description": "Una mezcla exótica y dulce de plátano, coco y chocolate, perfecta para consentirte."}
    Translation: {"name": "Caribbean (banana, coconut, chocolate)", "description": "An exotic and sweet blend of banana, coconut, and chocolate, perfect for indulging yourself."}

    Name: "Mocha"
    Description: {"description": "Una deliciosa combinación de café espresso, chocolate y leche cremosa, perfecta para los amantes del chocolate"}
    Translation: {"name": "Mocha", "description": "A delightful combination of espresso coffee, chocolate, and creamy milk, perfect for chocolate lovers."}

    Name: "Huevos Motuleños (2 huevos fritos sobre tortilla, salsa roja, jamón y plátano frito)"
    Description: {"description": "Un clásico desayuno yucateco con huevos fritos sobre tortilla, salsa roja, jamón y un toque dulce de plátano frito."}
    Translation: {"name": "Motuleños Eggs (2 fried eggs on tortilla, red sauce, ham, and fried plantain)", "description": "A classic Yucatecan breakfast with fried eggs on tortilla, red sauce, ham, and a sweet touch of fried plantain."}
    """

    TRANSLATION_PROMPT: str = """Translate the following name and description from Spanish to English. 
    Input:
    Name: "{name}"
    Description: "{description}"
    """

settings = Settings()



### Utils

In [76]:
def validate_response_length(text: str, max_length: int = settings.MAX_RESPONSE_LENGTH) -> bool:
    """
    Validates that the given response is less than or equal to the specified maximum length.

    Args:
        text (str): The response from the LLM.
        max_length (int): The maximum allowed length of the response. Default is 350.

    Returns:
        bool: True if the response length is valid, False otherwise.
    """
    return len(text) <= max_length

### Main (generate descriptions)

In [77]:
data = import_data(settings.FILE_PATH)

if not validate_data_structure(data=data):
    raise Exception("Please fix identified errors in the provided file.")

openai_client = get_openai_client()

categories = data.get(settings.CATEGORIES_LIST_FIELD, [])
for category in tqdm(categories, desc="Processing Categories"):
    category_name = category[settings.CATEGORY_FIELD]
    items = category.get(settings.ITEMS_LIST_FIELD, [])
    for item in tqdm(items, desc=f"Processing Items in {category_name}", leave=False):
        item_name = item[settings.ITEM_NAME_FIELD]
        try:
            description_json = generate_response(
                prompt=settings.DESCRIPTION_GENERATION_PROMPT.format(category=category_name, name=item_name),
                system_prompt=settings.DESCRIPTION_GENERATION_SYSTEM_PROMPT,
                openai_client=openai_client
            )

            description = json.loads(description_json)["description"]
            if not validate_response_length(text=description):
                raise Exception(f"Generated description is too long: {len(description)}")
            
            if description == "Unpredictable":
                print("Got uncertain position")
                
        except (KeyError, json.JSONDecodeError) as e:
            print(f"Error processing item '{item_name}' in category '{category_name}': {e}")
            description = "Description unavailable"
        except Exception as e:
            print(f"Error processing item '{item_name}' in category '{category_name}': {e}")
            description = "Description unavailable"

        item[settings.ITEM_DESCRIPTION_FIELD] = description
        
save_json(data=data, output_file_path=settings.RESULT_DESCRIPTIONS_FILE_PATH)

Processing Categories:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Items in Cafés:   0%|          | 0/6 [00:00<?, ?it/s]

Got uncertain position
Got uncertain position


Processing Items in Bebidas frías:   0%|          | 0/4 [00:00<?, ?it/s]

Processing Items in Jugos Frescos:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Items in Smoothies:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Items in Desayunos:   0%|          | 0/8 [00:00<?, ?it/s]

Processing Items in A cualquier hora:   0%|          | 0/14 [00:00<?, ?it/s]

Data successfully saved to /mnt/c/Users/Games/Desktop/beep-dev/prompting-test/data/solution_2_descriptions.json


LLM была неуверена в двух позициях меню в разделе Кофе:
- Coffee
- Doble

Если с Doble человеку будет понятно, что это двойной эспрессо (тк у него есть контекст, что прошлая позиция - эспрессо), то с Coffee возникнут проблемы и у обычного человека

### Main (translate)

In [78]:
def translate_data(data, target_language="en"):
    categories = data.get(settings.CATEGORIES_LIST_FIELD, [])
    for category in tqdm(categories, desc="Processing Categories"):
        category_name = category[settings.CATEGORY_FIELD]
        category[settings.CATEGORY_FIELD] = translate(category_name, target_language)

        items = category.get(settings.ITEMS_LIST_FIELD, [])
        for item in tqdm(items, desc=f"Processing Items in {category_name}", leave=False):
            translation_json = generate_response(
                prompt=settings.TRANSLATION_PROMPT.format(name=item[settings.ITEM_NAME_FIELD], description=item[settings.ITEM_DESCRIPTION_FIELD]),
                system_prompt=settings.TRANSLATION_SYSTEM_PROMPT,
                openai_client=openai_client
            )

            translation = json.loads(translation_json)
            item[settings.ITEM_NAME_FIELD] = translation["name"]
            item[settings.ITEM_DESCRIPTION_FIELD] = translation["description"]

    return data

translated_data = translate_data(data, target_language="en")
save_json(data=translated_data, output_file_path=settings.RESULT_TRANSLATION_FILE_PATH)

Processing Categories:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Items in Cafés:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Items in Bebidas frías:   0%|          | 0/4 [00:00<?, ?it/s]

Processing Items in Jugos Frescos:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Items in Smoothies:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Items in Desayunos:   0%|          | 0/8 [00:00<?, ?it/s]

Processing Items in A cualquier hora:   0%|          | 0/14 [00:00<?, ?it/s]

Data successfully saved to /mnt/c/Users/Games/Desktop/beep-dev/prompting-test/data/solution_2_translation.json


# Discussion

## Solution 2 Conclusion
Второе решение продемонстрировало прирост в качестве генерации описаний и перевода всего меню, все проблемы, выявленные при простом подходе были решены.

Однако, появились дополнительные нюансы, а именно, в двух позициях LLM не была уверена и не смогла сгенерировать описания основываясь на разделе меню и наименовании позиции, в результате чего, им было проставлено описания: "Unpredictable". Можно много рассуждать о целесообразности такой защиты в тестовом задании, но если воспринимать задание как реальную продуктовую систему, мне это кажется более чем оправданным, тк вопрос времени, когда придут позиции, в которых модель будет галюцинировать.

Такие позиции должны быть переданы на ручной анализ, в результате которого может быть несколько вариантов:
1. LLM ошиблась, позиции можно однозначно трактовать и генерировать к ним описание
2. LLM права, к данной позиции невозможно дать корректное описание на основании предоставленных данных

В первом случае можно улучшить промпт, добавив валидное описание для проблемной позиции, во втором варианте сложнее, решение зависит от желаний бизнеса, либо мы хотим гадать и нам некритично дать некорректное описание, либо расширяем набор входных данных (добавляем, например, состав блюда), либо одно из огромного множества альтернативных решений.

Касательно перевода с использованием LLM, безусловно работает лучше обычных переводчиков, но на реальной бизнесовой задаче, я бы задумался о целесообразности few-shot подхода и провел несколько тестов, чтобы выявить, есть ли от него значительный импакт.

## Further Improvements
Раскручивать идею дальше можно сколько угодно, вопрос в желании бизнеса, важности задачи (мало ли это core продукт), потенциально логичным было бы добавить подход к оценке, можно собрать валидационный датасет с эталонными описаниями, использовать их для примеров в few-shot, а также проводить на них замеры качества обновленной системы. Для оценки логичнее всего было бы использовать SOTA проприетарные модельки в качестве "LLM as a judge". Также, стоит отметить, что при таком подходе очень желательно добиться детерменированного результата от LLM (если ходить в openai, необходимо выставить нулевую температуру, добавить сид и следить за фингерпринтом ответа).
